In [1]:
import tensorflow as tf
import numpy as np
na = np.newaxis

import matplotlib.pyplot as plt

# 1. Inputs:
Domain $X = \mathbb{R}^R$ ($R$-Dimensional)
$T \subset X$


### Fixed:

#### General:
- R
- Tlims: (Rx2)
- Data D: (NxR) (all points in T)

#### Inducing points:
    -> fix nbr M 
    -> Z: (MxR) location
    -> u: (M) (each sample of function values at the inducing points is M dimensional) 

#### Hyperparameters ($\Theta$)
    -> fixed at first, might become be optimized later as well
$\Theta = (\gamma, \alpha_1,...,  \alpha_R, \overline{u})$


### Parameters:
variational dist at inducing points u: q(u) = N (u;m,S)

m: (M)

S: (MxM)

In [ ]:
# pseudo stuff, no working code!!!

# constants:
# - R, T, M
# - D (NxR)
# - Z (MxR)
# - Theta values (gamma, alphas, ustrich)

# placeholders:
# - m (M)
# - S (MxM)


# kernel stuff:

def kernel_function(X, Y):
    return K_XY

# kernels to compute: K_zz, K_zd, trace(K_dd)

def lower_bound(D, m, S, Theta, T):
    K_zz = ...
    K_zz_inv = ...
    
    return - region_integral(m, S, K_zz_inv, Z, Theta, T) + datapoint_expectations(D, m, S, K_zz_inv, Theta) - kl_term(m,S,K_zz, K_zz_inv, Theta) 

def kl_term(m, S, K_zz, K_zz_inv, Theta):
    return scalar_value_node

def datapoint_expectations(D, m, S, K_zz_inv, Theta):
    
    k_zd = ...
    k_dd = ...
    
    musqare_N = ...
    sigsquare_N = ...
    
    C = 0.57721... #Euler-Masceroni constant
    
    lookup values = ... # problem: how to implement lookup table
    
    return scalar_value_node

def region_integral(m, S, K_zz_inv, Z, Theta, T):
    Psi = ... (MxM) 
    return scalar_value_node

# Tensorflow Test

In [2]:
# inducing point locations
Z = np.linspace(0, 10, 11)[:,na]

In [3]:
# https://github.com/GPflow/GPflow/issues/439
def tf_tril_indices(N, k=0):
    M1 = tf.tile(tf.expand_dims(tf.range(N), axis=0), [N,1])
    M2 = tf.tile(tf.expand_dims(tf.range(N), axis=1), [1,N])
    mask = (M1-M2) >= -k
    ix1 = tf.boolean_mask(M2, tf.transpose(mask))
    ix2 = tf.boolean_mask(M1, tf.transpose(mask))
    return ix1, ix2

In [4]:
def ard_kernel(X1, X2, gamma=1., alphas=None):
    # X1:  (n1 x d)
    # X2:  (n2 x d)
    # out: (n1 x n2
    
    with tf.name_scope('ard_kernel'):
        if alphas is None:
            alphas = tf.ones([tf.shape(X1)[1]])
        return gamma * tf.reduce_prod(tf.exp(- (tf.expand_dims(X1, 1) - tf.expand_dims(X2, 0))**2 / (2 * tf.expand_dims(tf.expand_dims(alphas, 0), 0))), axis=2) 

def kl_term(m, S, K_zz, K_zz_inv, u_ovln):
    mean_diff = (u_ovln * tf.ones([tf.shape(Z_ph)[0]]) - m)
    first  = tf.trace(tf.matmul(K_zz_inv, S))
    second = tf.log(tf.norm(K_zz) / tf.norm(S))
    third  = tf.to_float(tf.shape(m)[0])
    fourth = tf.reduce_sum(tf.multiply(tf.reduce_sum(tf.multiply(mean_diff, tf.transpose(K_zz_inv)), axis=1) , mean_diff))
    return 0.5 * (first  - second - third + fourth)

In [5]:
tf.reset_default_graph()

Z_ph = tf.placeholder(tf.float32, [None, None], name='inducing_point_locations')
u_ph = tf.placeholder(tf.float32, [],           name='inducin_point_mean')
n_ph = tf.placeholder(tf.int32,   [],           name='number_samples')


num_inducing_points = 11# tf.shape(Z_ph)[0] TODO: was da tun?

# mean
m_init = tf.zeros([num_inducing_points])
m = tf.Variable(m_init)

# vectorized version of covariance matrix S (ensure valid covariance matrix)
vech_size   = (num_inducing_points * (num_inducing_points+1)) / 2 
vech_indices= tf.transpose(tf_tril_indices(num_inducing_points))
L_vech_init = tf.ones([vech_size])

L_vech = tf.Variable(L_vech_init)

L = tf.sparse_to_dense(vech_indices, [num_inducing_points, num_inducing_points], L_vech)
S = tf.matmul(L, tf.transpose(L))

K_zz  = ard_kernel(Z_ph, Z_ph)
K_zz_inv = tf.matrix_inverse(K_zz)

with tf.name_scope('approximation'):
    kl_term_op = kl_term(m, S, K_zz, K_zz_inv, u_ph)
    tf.summary.scalar('kl_div', kl_term_op)
    
with tf.name_scope('optimization'):
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(kl_term_op)

with tf.name_scope('prior_sampling'):
    cov  = K_zz
    mean = u_ph * tf.ones([num_inducing_points])
    ind_point_dist = tf.contrib.distributions.MultivariateNormalFullCovariance(mean, cov)
    samples = ind_point_dist.sample(n_ph)
    
merged = tf.summary.merge_all()

In [9]:
max_iterations = 5

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('logs', sess.graph)
    
    for i in range(max_iterations):
        _, summary, kl, m_val, s_val = sess.run([train_step, merged, kl_term_op, m, S], feed_dict={Z_ph:Z, u_ph:0.})
        writer.add_summary(summary, i)
        print('------------------------')
        print(kl)
        print(m_val)
        print(s_val)

------------------------
16.9261
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[[  1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.]
 [  1.   2.   2.   2.   2.   2.   2.   2.   2.   2.   2.]
 [  1.   2.   3.   3.   3.   3.   3.   3.   3.   3.   3.]
 [  1.   2.   3.   4.   4.   4.   4.   4.   4.   4.   4.]
 [  1.   2.   3.   4.   5.   5.   5.   5.   5.   5.   5.]
 [  1.   2.   3.   4.   5.   6.   6.   6.   6.   6.   6.]
 [  1.   2.   3.   4.   5.   6.   7.   7.   7.   7.   7.]
 [  1.   2.   3.   4.   5.   6.   7.   8.   8.   8.   8.]
 [  1.   2.   3.   4.   5.   6.   7.   8.   9.   9.   9.]
 [  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  10.]
 [  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.]]
------------------------
16.9261
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[[  1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.]
 [  1.   2.   2.   2.   2.   2.   2.   2.   2.   2.   2.]
 [  1.   2.   3.   3.   3.   3.   3.   3.   3.   3.   3.]
 [  1.   2.   3.   4.   4.   

### example sampling:

In [ ]:
# sampling
num_samples = 10

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    writer = tf.summary.FileWriter('logs', sess.graph)
    sample_res, mean_res, cov_res = sess.run([samples, mean, cov], feed_dict={Z_ph:Z, u_ph:0., n_ph:num_samples})

In [ ]:
for i in range(num_samples):
    plt.plot(Z, sample_res[i])
plt.show()